<a href="https://colab.research.google.com/github/Madushani-Weerasekara/Colloborative-Filtering-Recommendation-Engine/blob/main/nlp_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import needed libraries

In [5]:
# import necesary libraries
import pandas as pd
import numpy as np
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# mount google drive

In [6]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# load the dataset

In [8]:
# define the file path
file_path = '/content/drive/MyDrive/Colab Notebooks/GitHub Projects/Colloborative-Filtering-Recommendation-Engine/Data/user_reviews_edited.csv'

In [9]:
# lood the dataset
df = pd.read_csv(file_path)


In [10]:
df.head(5)

,Review ID,User ID,Location,Rating,Review
0,1,1,Yala,5,Amazing wildlife experience
1,2,1,Horton Plains,4,Beautiful hiking trails
2,3,2,Kandy,4,Rich history and culture
3,4,2,Sigiriya,5,Stunning ancient rock fortress
4,5,3,Hikkaduwa,3,Nice beaches but can get crowded


In [12]:
df.shape

(10, 5)

In [13]:
df.isnull().sum()

,0
Review ID,0
User ID,0
Location,0
Rating,0
Review,0


In [14]:
df.columns

Index(['Review ID', 'User ID', 'Location', 'Rating', 'Review '], dtype='object')

In [15]:
df.drop(columns='Rating')



,Review ID,User ID,Location,Review
0,1,1,Yala,Amazing wildlife experience
1,2,1,Horton Plains,Beautiful hiking trails
2,3,2,Kandy,Rich history and culture
3,4,2,Sigiriya,Stunning ancient rock fortress
4,5,3,Hikkaduwa,Nice beaches but can get crowded
5,6,3,Unawatuna,Lovely beach and great food
6,7,4,Yala,Incredible safari and animal sightings
7,8,4,Nuwara Eliya,Beautiful scenery and tea plantations
8,9,5,Sinharaja,A great place for nature lovers
9,10,5,Pinnawala,Interesting elephant orphanage
